In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle 
from tabulate import tabulate

In [2]:
ga_path = '../pickles/ga/'
kbga_path = '../pickles/kbga/corrected/'

In [3]:
ga_files = ['n_run_az_100fc_ts.pkl', 
            "n_run_az_ga_100fc_copt_avg.pkl"]

In [5]:
kbga_files = ['n_run_az_kbga_es50_100.pkl',
              'n_run_az_kbga_vp.pkl',
              'n_run_az_kbga_es10_100.pkl',
              'n_run_az_kbga_nokbps.pkl', 
              'n_run_az_kbga_cmp.pkl']

In [6]:
def average_best_fitness_value(az, imdb, yelp):
    cols = ["gen_"+str(i) for i in [1, 25, 50, 75, 100]]
    index_col = ["Amazon", "IMDB", "Yelp", "Average"]
    
    data = [
        az.iloc[-1, [0, 24, 49, 74, 99]],
        imdb.iloc[-1, [0, 24, 49, 74, 99]],
        yelp.iloc[-1, [0, 24, 49, 74, 99]],
        ]
    
    abfv = pd.DataFrame(data, columns=cols)
    abfv = abfv.append(abfv.mean(), ignore_index=True)

    abfv.index = index_col
    abfv.index.name = "Datasets"
    
    return abfv

In [7]:
def average_best_of_generation(az, imdb, yelp):
    cols = ["1-25", "26-50", "51-75", "76-100"]
    index_col = ["Amazon", "IMDB", "Yelp", "Average"]
    
    data = [
        [az.iloc[-1, i-25:i].mean() for i in range(25, 101, 25)],
        [imdb.iloc[-1, i-25:i].mean() for i in range(25, 101, 25)],
        [yelp.iloc[-1, i-25:i].mean() for i in range(25, 101, 25)],
    ]

    abog = pd.DataFrame(data, columns=cols)
    abog = abog.append(abog.mean(), ignore_index=True)

    abog.index = index_col
    abog.index.name = "Datasets"
    return abog

In [8]:
def optimization_accuracy(az, imdb, yelp):
    cols = ["gen_"+str(i) for i in [1, 25, 50, 75, 100]]
    index_col = ["Amazon", "IMDB", "Yelp", "Average"]
    
    mins = min(base_az.min().to_list())
    maxs = max(base_az.max().to_list())
    
    data = [
        list(map(lambda i: (i-mins)/(maxs-mins), az.iloc[-1, [0, 24, 49, 74, 99]])),
        list(map(lambda i: (i-mins)/(maxs-mins), imdb.iloc[-1, [0, 24, 49, 74, 99]])),
        list(map(lambda i: (i-mins)/(maxs-mins), yelp.iloc[-1, [0, 24, 49, 74, 99]])),
    ]

    oa = pd.DataFrame(data, columns=cols)
    oa = oa.append(oa.mean(), ignore_index=True)

    oa.index = index_col
    oa.index.name = "Datasets"
    return oa

In [158]:
def evolutionary_leap(base, runs):
    leap = [[0, 0, 0, 0, 0]]
    for i in range(1, 100):
        # print(ga_az.iloc[:, i] - ga_az.iloc[:, i-1])
        leap.append(list(map(lambda x: 1 if x != 0 else 0, base.iloc[:-1, i] - base.iloc[:-1, i-1])))
    leap = np.transpose(leap)

    indexes = ["run_"+str(i) for i in range(1, 5+1)]
    leap_df = pd.DataFrame(leap, columns=base.columns[:-1])
    leap_df.index = indexes

    data = {
        "gen_25": leap_df.iloc[:, :25].sum(axis=1),
        "gen_50": leap_df.iloc[:, 25:50].sum(axis=1),
        "gen_75": leap_df.iloc[:, 50:75].sum(axis=1),
        "gen_100": leap_df.iloc[:, 75:].sum(axis=1)
    }
    
    leap_count = pd.DataFrame(data)
    leap_count = leap_count.append(leap_count.mean(), ignore_index=True)
    
    indexes = ["run_"+str(i) for i in range(1, runs+1)] + ['Average']
    leap_count.index = indexes
    leap_count.index.name = 'runs'
    
    return leap_count


def likelihood_of_evolution_leap(az, imdb, yelp, runs):
    cols = ["gen_"+str(i) for i in [25, 50, 75, 100]]
    index_col = ["Amazon", "IMDB", "Yelp", "Average"]

    az_leaps = evolutionary_leap(az, runs)
    imdb_leaps = evolutionary_leap(imdb, runs)
    yelp_leaps = evolutionary_leap(yelp, runs)

    data = [
        az_leaps.iloc[-1]/runs,
        imdb_leaps.iloc[-1]/runs,
        yelp_leaps.iloc[-1]/runs,
    ]

    el = pd.DataFrame(data, columns=cols)
    el = el.append(el.mean(), ignore_index=True)

    el.index = index_col
    el.index.name = "Datasets"

    return el

In [160]:
def probability_of_convergence(az, imdb, yelp, success_thresh, runs):
    az_count = 0
    imdb_count = 0
    yelp_count = 0

    for i in range(runs):
        if az.iloc[i, -2] >= success_thresh:
            az_count += 1
        if imdb.iloc[i, -2] >= success_thresh:
            imdb_count += 1
        if yelp.iloc[i, -2] >= success_thresh:
            yelp_count += 1

    cols = ["P"]
    index_col = ["Amazon", "IMDB", "Yelp", "Average"]

    data = [
        az_count/runs,
        imdb_count/runs,
        yelp_count/runs
    ]

    pc = pd.DataFrame(data, columns=cols)
    pc = pc.append(pc.mean(), ignore_index=True)

    pc.index = index_col
    pc.index.name = "Datasets"

    return pc

In [167]:
def function_evaluations(base, success_thresh, runs):
    evolutions = 0

    mask = base.iloc[:, :-1] >= success_thresh
    for i in range(runs):
        try:
            # print(base[mask].iloc[i].dropna())
            evolutions += int(base[mask].iloc[i].dropna().index[0].split("_")[1]) 
        except:
            evolutions += 0

    return evolutions


def average_no_of_function_evaluations(az, imdb, yelp, success_thresh, runs):
    az_eval = function_evaluations(az, success_thresh, runs)
    imdb_eval = function_evaluations(imdb, success_thresh, runs)
    yelp_eval = function_evaluations(yelp, success_thresh, runs)

    cols = ["AFES"]
    index_col = ["Amazon", "IMDB", "Yelp", "Average"]

    data = [
        az_eval/runs,
        imdb_eval/runs,
        yelp_eval/runs
    ]

    afes = pd.DataFrame(data, columns=cols)
    afes = afes.append(afes.mean(), ignore_index=True)

    afes.index = index_col
    afes.index.name = "Datasets"

    return afes

In [185]:
def successful_performance(az, imdb, yelp, success_thresh, runs):
    afes = average_no_of_function_evaluations(az, imdb, yelp, success_thresh, runs)
    p = probability_of_convergence(az, imdb, yelp, success_thresh, runs)

    cols = ["SP"]
    index_col = ["Amazon", "IMDB", "Yelp", "Average"]

    data = [
        afes.iloc[0, 0]/p.iloc[0, 0],
        afes.iloc[1, 0]/p.iloc[1, 0],
        afes.iloc[2, 0]/p.iloc[2, 0]
    ]

    sp = pd.DataFrame(data, columns=cols)
    sp = sp.append(sp.mean(), ignore_index=True)

    sp.index = index_col
    sp.index.name = "Datasets"

    return sp

In [213]:
from scipy import stats

def one_tailed_t_test(abfv, abog, oa, el):
    h0 = False
    alpha = 0.05


    t_value, p_value = stats.ttest_1samp(abfv.iloc[:-1, :5], abfv.iloc[:-1, 5:])
    p_df = pd.DataFrame(p_value).astype()
    print(p_df)
        

In [214]:
one_tailed_t_test(abfv, abog, oa, el)

     0             1    2             3    4
0  0.0  4.913087e-31  0.0  4.008179e-31  0.0
1  0.0  4.913087e-31  0.0  4.008179e-31  0.0
2  0.0  4.913087e-31  0.0  4.008179e-31  0.0


In [26]:
def tabulate_runs(save_path, runs):
    with open(save_path, 'rb') as gf:
        data = pickle.load(gf)
        chromos = []
        scores = []
        exec_time = []
        for run in data:
            chromos.append(run[0])
            scores.append(run[1])
            exec_time.append(run[2])

    df = pd.DataFrame()
    cols = ["gen_" + str(i) for i in range(1, 101)]
    df[cols] = pd.DataFrame(scores)

    df = pd.concat([df, pd.DataFrame({'exec_time': exec_time})], axis=1)
    df = df.append(df.mean(), ignore_index=True)

    indexes = ["run_"+str(i) for i in range(1, runs+1)] + ['Average']
    df.index = indexes
    df.index.name = 'runs'
    
    return df

In [164]:
runs = 5
success_thresh = 0.8

ga_az = tabulate_runs(ga_path+"corrected/"+ga_files[0], runs)
kbga_az = tabulate_runs(kbga_path+kbga_files[1], runs)

base_az = pd.concat([ga_az.iloc[:, :-1], kbga_az.iloc[:, :-1]], axis=1)
base_az

,gen_1,gen_2,gen_3,gen_4,gen_5,gen_6,gen_7,gen_8,gen_9,gen_10,...,gen_91,gen_92,gen_93,gen_94,gen_95,gen_96,gen_97,gen_98,gen_99,gen_100
runs,,,,,,,,,,,,,,,,,,,,,
run_1,0.6240,0.6240,0.6600,0.660,0.6600,0.6680,0.676,0.6800,0.6800,0.6840,...,0.9000,0.9000,0.9000,0.9000,0.9000,0.9000,0.9000,0.9000,0.9000,0.900
run_2,0.6240,0.6520,0.6640,0.668,0.6960,0.6960,0.696,0.7040,0.7040,0.7120,...,0.8840,0.8880,0.8880,0.8880,0.8880,0.8920,0.8920,0.8920,0.8920,0.892
run_3,0.6200,0.6440,0.6440,0.648,0.6480,0.6520,0.676,0.6760,0.6800,0.6880,...,0.8920,0.8920,0.8920,0.8920,0.8920,0.8920,0.8920,0.8920,0.8920,0.896
run_4,0.6680,0.6680,0.6680,0.668,0.6680,0.6760,0.676,0.6760,0.6880,0.6880,...,0.8960,0.8960,0.8960,0.8960,0.8960,0.8960,0.8960,0.8960,0.8960,0.896
run_5,0.6320,0.6400,0.6400,0.676,0.6760,0.6760,0.676,0.6760,0.6800,0.6800,...,0.8960,0.8960,0.8960,0.8960,0.8960,0.8960,0.8960,0.8960,0.8960,0.896
Average,0.6336,0.6456,0.6552,0.664,0.6696,0.6736,0.680,0.6824,0.6864,0.6904,...,0.8936,0.8944,0.8944,0.8944,0.8944,0.8952,0.8952,0.8952,0.8952,0.896


## Measures

In [13]:
abfv_ga = average_best_fitness_value(ga_az, ga_az, ga_az)
abfv_kbga = average_best_fitness_value(kbga_az, kbga_az, kbga_az)
abfv = pd.concat([abfv_ga, abfv_kbga], axis=1)
abfv

,gen_1,gen_25,gen_50,gen_75,gen_100,gen_1,gen_25,gen_50,gen_75,gen_100
Datasets,,,,,,,,,,
Amazon,0.6336,0.7248,0.7464,0.764,0.7696,0.6768,0.8368,0.8736,0.888,0.896
IMDB,0.6336,0.7248,0.7464,0.764,0.7696,0.6768,0.8368,0.8736,0.888,0.896
Yelp,0.6336,0.7248,0.7464,0.764,0.7696,0.6768,0.8368,0.8736,0.888,0.896
Average,0.6336,0.7248,0.7464,0.764,0.7696,0.6768,0.8368,0.8736,0.888,0.896


In [15]:
abog_ga = average_best_of_generation(ga_az, ga_az, ga_az)
abog_kbga = average_best_of_generation(kbga_az, kbga_az, kbga_az)
abog = pd.concat([abog_ga, abog_kbga], axis=1)
abog

,1-25,26-50,51-75,76-100,1-25,26-50,51-75,76-100
Datasets,,,,,,,,
Amazon,0.696384,0.738208,0.756352,0.766944,0.781792,0.859904,0.881696,0.892896
IMDB,0.696384,0.738208,0.756352,0.766944,0.781792,0.859904,0.881696,0.892896
Yelp,0.696384,0.738208,0.756352,0.766944,0.781792,0.859904,0.881696,0.892896
Average,0.696384,0.738208,0.756352,0.766944,0.781792,0.859904,0.881696,0.892896


In [16]:
oa_ga = optimization_accuracy(ga_az, ga_az, ga_az)
oa_kbga = optimization_accuracy(kbga_az, kbga_az, kbga_az)
oa = pd.concat([oa_ga, oa_kbga], axis=1)
oa

,gen_1,gen_25,gen_50,gen_75,gen_100,gen_1,gen_25,gen_50,gen_75,gen_100
Datasets,,,,,,,,,,
Amazon,0.048571,0.374286,0.451429,0.514286,0.534286,0.202857,0.774286,0.905714,0.957143,0.985714
IMDB,0.048571,0.374286,0.451429,0.514286,0.534286,0.202857,0.774286,0.905714,0.957143,0.985714
Yelp,0.048571,0.374286,0.451429,0.514286,0.534286,0.202857,0.774286,0.905714,0.957143,0.985714
Average,0.048571,0.374286,0.451429,0.514286,0.534286,0.202857,0.774286,0.905714,0.957143,0.985714


In [188]:
el_ga = likelihood_of_evolution_leap(ga_az, ga_az, ga_az, runs)
el_kbga = likelihood_of_evolution_leap(kbga_az, kbga_az, kbga_az, runs)
el = pd.concat([el_ga, el_kbga], axis=1)
el

,gen_25,gen_50,gen_75,gen_100,gen_25,gen_50,gen_75,gen_100
Datasets,,,,,,,,
Amazon,1.72,0.8,0.48,0.28,3.24,1.56,0.68,0.4
IMDB,1.72,0.8,0.48,0.28,3.24,1.56,0.68,0.4
Yelp,1.72,0.8,0.48,0.28,3.24,1.56,0.68,0.4
Average,1.72,0.8,0.48,0.28,3.24,1.56,0.68,0.4


In [166]:
pc_ga = probability_of_convergence(ga_az, ga_az, ga_az, success_thresh, runs)
pc_kbga = probability_of_convergence(kbga_az, kbga_az, kbga_az, success_thresh, runs)
pc = pd.concat([pc_ga, pc_kbga], axis=1)
pc

,P,P
Datasets,,
Amazon,0.2,1.0
IMDB,0.2,1.0
Yelp,0.2,1.0
Average,0.2,1.0


In [165]:
afes_ga = average_no_of_function_evaluations(ga_az, ga_az, ga_az, success_thresh, runs)
afes_kbga = average_no_of_function_evaluations(kbga_az, kbga_az, kbga_az, success_thresh, runs)
afes = pd.concat([afes_ga, afes_kbga], axis=1)
afes

,AFES,AFES
Datasets,,
Amazon,15.4,15.0
IMDB,15.4,15.0
Yelp,15.4,15.0
Average,15.4,15.0


In [187]:
successful_performance(ga_az, ga_az, ga_az, success_thresh, runs)
sp_ga = successful_performance(ga_az, ga_az, ga_az, success_thresh, runs)
sp_kbga = successful_performance(kbga_az, kbga_az, kbga_az, success_thresh, runs)
sp = pd.concat([sp_ga, sp_kbga], axis=1)
sp

,SP,SP
Datasets,,
Amazon,77.0,15.0
IMDB,77.0,15.0
Yelp,77.0,15.0
Average,77.0,15.0


## Save as CSV

In [215]:
with open('report.csv', 'wb') as rf:
    base_az.to_csv(rf)

with open('report.csv', 'ab') as rf:
    for i in [abfv, abog, oa, el]:
        i.to_csv(rf, header=True)